In [54]:
print("Why hello there")

Why hello there


In [580]:
import cv2
import pytesseract
import os
from os import walk
import pandas as pd
import re
from PIL import Image
from PIL.ExifTags import GPSTAGS, TAGS
import numpy as np

In [56]:
dir1 = os.getcwd() + "/ScreenShots"
filenames = next(walk(dir1), (None, None, []))[2]
filenames = sorted(filenames)

In [57]:
# DF FOR TIMESTAMPS ON FILES
timestampList = []
fileNameList = []
for file in filenames:
    try:
        fileNameList.append(file)
        file = dir1 + "/" + file
        image = Image.open(file)
        if image._getexif() == None:
            fileNamstList.pop(-1)
            print(f"{file} contains no exif data.")
        else:
            for tag, value in image._getexif().items():
                tag_name = TAGS.get(tag)
                if tag_name == "DateTimeOriginal":
                    timestampList.append(value)
    except IOError:
        fileNamstList.pop(-1)
        print("File format not supported!")
dfTuples = list(zip(fileNameList, timestampList))    
timeDF = pd.DataFrame(dfTuples, columns=["FileName", "Time"])

In [58]:
# GOES THROUGH ALL FILES
# READS TEXT FROM IMAGES, SORTS TO DELIVERY OFFERS AND DELIVERY COMPLETIONS
# SAVES TEXT AND FILE NAME APPROPRIATELY
def testFunc(files_to_process_list):
    offerCounter = completeCounter = driveCounter = thirdCounter = 0
    theList = []
    completionList = []
    offerFiles = []
    compFiles = []
    for file in files_to_process_list:
        #file to image
        img = cv2.imread(dir1 + "/" + file)
        #image to text
        data = pytesseract.image_to_string(img)
        #MARKETPLACE DELIVERY OFFER
        if("Guaranteed" in data):
            # The above text should only be found in marketplace delivery offers
            # We want to append the file to the list of offers
            # Limiting text to things after the $ excludes anything read from the map
            # Split the text data into lines and get rid of empty lines
            # If $ cannot be found, split the text into lines and add the entire list to be processed later
            offerFiles.append(file)
            offerCounter += 1
            if("$" in data):
                position = data.index("$")
                textsplit = data[position:].splitlines()
                while ("" in textsplit):
                    textsplit.remove("")
                theList.append(textsplit)
            else:
                data = data.splitlines()
                theList.append(data)
        #DELIVERY COMPLETIONS
        elif("Completed" in data):
            compFiles.append(file)
            img = cv2.imread(dir1 + "/" + file)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            data = pytesseract.image_to_string(thresh, lang='eng',config='--psm 6')
            # The above filters are applied to improve extraction quality
            completeCounter += 1
            # The below section works as before to exclude anything read from the map
            if("$" in data):
                position = data.index("$")
                textsplit = data[position:].splitlines()
                while("" in textsplit):
                    textsplit.remove("")
                completionList.append(textsplit)
            else:
                print("Error in", file, "$ not found")
                data = data.splitlines()
                completionList.append(data)
        #DRIVE DELIVERY OFFER
        elif("item" in data):
            # The interface has a slightly different format, so split the text into list of lines and process later
            offerFiles.append(file)
            driveCounter += 1
            data = data.splitlines()
            theList.append(data)                   
        else:
            print(file)
            print("ERROR WITH THIS FILE")
            thirdCounter += 1
    return offerCounter, completeCounter, driveCounter, thirdCounter, theList, completionList, offerFiles, compFiles

x = testFunc(filenames)
offers = x[0]
completes = x[1]
drives = x[2]
others = x[3]
offersList = x[4]
completionList = x[5]
offerFiles = x[6]
compFiles = x[7]

In [65]:
betterList = []
badList = []
# For every list of strings in the list of delivery offers
for x in offersList:
    indexA = 0
    indexB = 0
    startVal = endVal = 0
    DriveOrderFlag = 0
    # For every string in the list
    for y in x:
        # If Drive order, add string and break
        if "item" in y:
            betterList.append(x)
            DriveOrderFlag = 1
            break
        # If Marketplace order
        if "Guaranteed" in y:
            startVal = indexA
        if "Customer dropoff" in y:
            endVal = indexB
        indexA +=1
        indexB +=1
        if endVal != 0:
            betterList.append(x[startVal: endVal+1])
            break
    if(endVal == 0 and DriveOrderFlag != 1):
        print("Yikes, adding to awkward list")
        print(indexA, indexB, startVal, endVal)
        badList.append(x)

In [379]:
#print(betterList)
#print(badList)
#LSTM CHECKPOINT

In [380]:
weirdDFAttempt = pd.DataFrame(betterList)
weirdDFAttempt.fillna("", inplace=True)
# Convert list to DF, drop whitespace
# For every column, for every row
# If "DoorDash pay" found, it is used as an anchor from which other values are found
# Set the column values for column 0, 1, 3, and 4 for ease of later processing
for col in weirdDFAttempt:
    targetRow = targetCol = -1
    for index, value in weirdDFAttempt[col].items():
        if "DoorDash pay" in value:
            targetRow = index
            targetCol = col
            offerVal = weirdDFAttempt[targetCol-2][targetRow]
            mileVal = weirdDFAttempt[targetCol-4][targetRow]
            restVal = weirdDFAttempt[targetCol-6][targetRow]
            weirdDFAttempt.at[targetRow, 0] = offerVal + " Guaranteed"
            weirdDFAttempt.at[targetRow, 1] = mileVal
            weirdDFAttempt.at[targetRow, 3] = "Pickup"
            weirdDFAttempt.at[targetRow, 4] = restVal

In [381]:
# Function to convert $ string to float, 
# I should expand to allow for multiple columns, handle the large DF at end instead of after func def
#OKAY SO THIS DOESN'T CONVERT TO FLOAT ANYMORE IT JUST PASSES THE STRING VALUE OF THE DOLLAR AMOUNT
#DONT EVEN STRIP THE DOLLAR WE NEED IT FOR LATER
def dollarToFloat(df):
    index = 0
    for x in df[0]:
        if("$" in x):
            x = x.replace(" ", "")
            x = x.replace("+", "")
            a = x.index("$")
            b = x.index("Guaranteed")
            #x = x.replace("$", "")
            the_float = x[a:b]
            df.at[index, 0] = the_float
        index +=1
dollarToFloat(weirdDFAttempt)

In [382]:
# This looks for "mi" although technically "yards" can show up as an option, this would not handle that

# Look through columns given, in this case 1 and 2, extract the number and convert to float
# Due to image to text reads the value could be in either column, combine values
# Then for every string in the Series, if "mi" is present, use regex to find the value just before
# Convert that value to float, and place it back in the "Mileage" column


# so df["StackFlag"] = 0
# if "Additional" in x, df.at[itr, "StackFlag"] = 1
def mi2fl(df, columns):
    df["Mileage"] = df[columns[0]] + " " + df[columns[1]]
    df["Stacked Order"] = 0
    itr = 0
    for x in df["Mileage"]:
        if( "mi" in x):
            #Okay so \S+ is a word and \s+ is whitespace, so this means find 1 word appearing before mi from x
            match = re.search('(\S+\s+){1}(?=mi)', x)
            df.at[itr, "Mileage"] = float(match.group(0))
            if ("Additional" in x):
                df.at[itr, "Stacked Order"] = 1
        else:
            df.at[itr, "Mileage"] = "ERROR"
        itr +=1       
mi2fl(weirdDFAttempt, [1,2])

In [383]:
weirdDFAttempt

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,Mileage,Stacked Order
0,$13.50,5.5 mi,Deliver by 1:45 PM,a Pickup,China Fusion,f Customer dropoff,,,,,...,,,,,,,,,5.5,0
1,$9.00,3.8 mi,Deliver by 2:05 PM,a Pickup,Hooked on Sushi,f Customer dropoff,,,,,...,,,,,,,,,3.8,0
2,$6.50,4.6 mi,Deliver by 2:26 PM,@ Pickup,Chick-fil-A,f8 Customer dropoff,,,,,...,,,,,,,,,4.6,0
3,$11.50,10.8 mi,Deliver by 3:31PM,Pickup,Sushi Yukiya,Retail Pickup,7-Eleven,f Customer dropoff,,,...,,,,,,,,,10.8,0
4,$6.75,3.4 mi,Deliver by 3:38 PM,a Pickup,Starbucks,f Customer dropoff,,,,,...,,,,,,,,,3.4,0
5,$10.75,4.2 mi,Deliver by 4:17 PM,a Pickup,Pizza on tap,f Customer dropoff,,,,,...,,,,,,,,,4.2,0
6,$7.00,Additional 3.7 mi,Deliver by 4:34 PM,a Pickup,Popeyes Louisiana Kitchen,f Customer dropoff,,,,,...,,,,,,,,,3.7,1
7,$19.75,10.5 mi,Deliver by 5:37 PM,a Pickup,Upper Crust Pizza,f Customer dropoff,,,,,...,,,,,,,,,10.5,0
8,$4.75,1.6 mi,Deliver by 6:11PM,a Pickup,Wendy's,f Customer dropoff,,,,,...,,,,,,,,,1.6,0
9,$6.75,2 items e 1.5 mi total,Decline,Pickup,Advance Auto Parts,oO Medical Center,,,,© The|Home Depot,...,2 items e 1.5 mi total,,$6.75,,DoorDash pay,,,,1.5,0


In [384]:
weirdDFAttempt = weirdDFAttempt.rename(columns={0: "Dollars"})

In [385]:
# A few things complicate these reads
# The restaurant will always show in [4] OR [5]....so far
# For natural stacks, the second restaurant will appear in [6] or [7]
# This will grab the first restaurant, and set the second to either be "" or the name of the second restaurant
# Or it will save the names as ERROR
def getRestNames(df):
    restNames = []
    rest2Names = []
    for index, row in df.iterrows():
        restName = ""
        rest2 = ""
        if("Pickup" in row[3]):
            restName = row[4].replace("| ", "")
            restNames.append(restName)
            if("Pickup" in row[5]):
                rest2 = row[6]
                rest2Names.append(rest2)
            else:
                rest2Names.append(rest2)
        elif("Pickup" in row[4]):
            restName = row[5].replace("| ", "")
            restNames.append(restName)
            if("Pickup" in row[6]):
                rest2 = row[7]
                rest2Names.append(rest2)
            else:
                rest2Names.append(rest2)
        else:
            print("error")
            restNames.append("ERROR")
            rest2Names.append("ERROR")
    df["RestNames"] = restNames
    df["Rest 2"] = rest2Names 
getRestNames(weirdDFAttempt)

error


In [386]:
# Extract only the useful columns to new DF
offerDF = weirdDFAttempt[["Dollars", "Mileage", "RestNames", "Rest 2", "Stacked Order"]]

In [387]:
# In this method, we immediately take the values into a DF
# There isn't really a good reason for two different methods, I was just tired on two different days
# I think the first process should be refactored using this method if possible
# Or the speed of the process for old vs refactored could be found at least
cDF = pd.DataFrame(completionList)
cDF.fillna("", inplace=True)

In [388]:
#so index is 0-26, row is a series with each element in the series being one column in order
#We dont need peak pay conditional as its own giant branch, can just have a flag, if peak pay peakFlag

for index1, row in cDF.iterrows():
    happyFlag = False
    stackFlag = False
    for index2, value in row.items():
        if("Customer Tips" in value):
            happyFlag = True
            ############################ PEAK PAY CONDITIONAL ##############################
            if("Peak Pay" in cDF[index2-1][index1]):
                basePay = cDF[index2-2][index1]
                peakPay = cDF[index2-1][index1]
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.0"
                    restName = restName[:restName.index("-")]
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
            ################################## STANDARD CONDITIONAL ##################################
            else:
                basePay = cDF[index2-1][index1]
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.0"
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
                    if(")" in rest2):
                        if not ("(" in rest2):
                            rest2 = ""
                            stackFlag = False               
    if not happyFlag:
            print("ERROR AT ROW:", index1)

ERROR AT ROW: 19


In [389]:
completedDF = pd.DataFrame(columns = ["Base Pay", "Tip", "Peak Pay", 
                                          "Restaurant Name", "Restaurant Name 2", "Tip 2", "Total"])
#index 1 is row #
#index 2 is col #
#happyFlag checks for the existence of Customer Tips in the row
# ----- but should be errorFlag and cover all problems
# ----- Cust Tips is important since we use it as an anchor point
for index1, row in cDF.iterrows():
    # FLAGS #
    happyFlag = False
    stackFlag = False
    # BASE CASE ELEMENT ATTRIBUTES
    basePay = 0
    tipPay = 0
    peakPay = 0
    totalPay = 0
    rest2 = "N/A"
    tip2 = "N/A"
    for index2, value in row.items():
        if("Customer Tips" in value):
            happyFlag = True
            ############################ PEAK PAY INDEXING ##############################
            if("Peak Pay" in cDF[index2-1][index1]):
                basePay = cDF[index2-2][index1]
                peakPay = cDF[index2-1][index1]
                if ("$" in peakPay):
                    peakPay = peakPay[peakPay.index("$"):] 
                else:
                    peakPay = "ERROR"
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.00"
                    restName = restName[:restName.index("-")]
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if ("$" in rest2):
                        tip2 = rest2[rest2.index("$"):]
                        rest2 = rest2[:rest2.index("$")]
                    else:
                        tip2 = "$0.00"
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
                    if(")" in rest2):
                        if not ("(" in rest2):
                            rest2 = "N/A"
                            tip2 = "N/A"
                            stackFlag = False            
            ################################## STANDARD INDEXING ##################################
            else:
                peakPay = "$0.00"
                basePay = cDF[index2-1][index1]
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.00"
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if ("$" in rest2):
                        tip2 = rest2[rest2.index("$"):]
                        rest2 = rest2[:rest2.index("$")]
                    else:
                        tip2 = "$0.00"
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
                    if(")" in rest2):
                        if not ("(" in rest2):
                            rest2 = "N/A"
                            tip2 = "N/A"
                            stackFlag = False
    if not happyFlag:
        newRowList = [0, 0, 0, 0, 0, 0, 0]
        completedDF.loc[len(completedDF)] = newRowList
    else:
        newRowList = [basePay, tipPay, peakPay, restName, rest2, tip2, total]
        completedDF.loc[len(completedDF)] = newRowList

In [390]:
#What if we kept Filename, then dropped it


#offerDF["Offer File Name"] = offerFiles
offerDF["FileName"] = offerFiles
display(offerDF)
offerDF = offerDF.join(timeDF.set_index('FileName'), on="FileName")
display(offerDF)
offerDF = offerDF.rename(columns={"Time" : "Offer Timestamp"})
offerDF = offerDF.drop(["FileName"], axis=1)

/tmp/ipykernel_7900/905133280.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offerDF["FileName"] = offerFiles


,Dollars,Mileage,RestNames,Rest 2,Stacked Order,FileName
0,$13.50,5.5,China Fusion,,0,IMG_2279.PNG
1,$9.00,3.8,Hooked on Sushi,,0,IMG_2281.PNG
2,$6.50,4.6,Chick-fil-A,,0,IMG_2284.PNG
3,$11.50,10.8,Sushi Yukiya,7-Eleven,0,IMG_2289.PNG
4,$6.75,3.4,Starbucks,,0,IMG_2298.PNG
5,$10.75,4.2,Pizza on tap,,0,IMG_2303.PNG
6,$7.00,3.7,Popeyes Louisiana Kitchen,,1,IMG_2304.PNG
7,$19.75,10.5,Upper Crust Pizza,,0,IMG_2309.PNG
8,$4.75,1.6,Wendy's,,0,IMG_2319.PNG
9,$6.75,1.5,Advance Auto Parts,,0,IMG_2321.PNG


,Dollars,Mileage,RestNames,Rest 2,Stacked Order,FileName,Time
0,$13.50,5.5,China Fusion,,0,IMG_2279.PNG,2023:06:24 13:17:06
1,$9.00,3.8,Hooked on Sushi,,0,IMG_2281.PNG,2023:06:24 13:44:10
2,$6.50,4.6,Chick-fil-A,,0,IMG_2284.PNG,2023:06:24 14:01:32
3,$11.50,10.8,Sushi Yukiya,7-Eleven,0,IMG_2289.PNG,2023:06:24 14:32:10
4,$6.75,3.4,Starbucks,,0,IMG_2298.PNG,2023:06:24 15:15:23
5,$10.75,4.2,Pizza on tap,,0,IMG_2303.PNG,2023:06:24 15:52:54
6,$7.00,3.7,Popeyes Louisiana Kitchen,,1,IMG_2304.PNG,2023:06:24 15:55:57
7,$19.75,10.5,Upper Crust Pizza,,0,IMG_2309.PNG,2023:06:24 16:49:34
8,$4.75,1.6,Wendy's,,0,IMG_2319.PNG,2023:06:24 17:59:33
9,$6.75,1.5,Advance Auto Parts,,0,IMG_2321.PNG,2023:06:24 18:11:48


In [391]:
#timeDF

In [392]:
#completedDF["Completed File Name"] = compFiles
completedDF["FileName"] = compFiles
#display(completedDF)
#completedDF = completedDF.join(timeDF)
completedDF = completedDF.join(timeDF.set_index('FileName'), on="FileName")
#display(offerDF)
completedDF = completedDF.rename(columns={"Time" : "Completed Timestamp"})
completedDF = completedDF.drop(["FileName"], axis=1)

In [393]:
totalDF = completedDF.join(offerDF)
totalDF

,Base Pay,Tip,Peak Pay,Restaurant Name,Restaurant Name 2,Tip 2,Total,Completed Timestamp,Dollars,Mileage,RestNames,Rest 2,Stacked Order,Offer Timestamp
0,$6.50,$7.00,$0.00,China Fusion,N/A,N/A,$13.50,2023:06:24 13:38:54,$13.50,5.5,China Fusion,,0,2023:06:24 13:17:06
1,$4.00,$5.00,$0.00,Hooked on Sushi (Carlsbad),N/A,N/A,$9.00,2023:06:24 13:59:00,$9.00,3.8,Hooked on Sushi,,0,2023:06:24 13:44:10
2,$5.50,$1.00,$0.00,Chick-fil-A (01507),N/A,N/A,$6.50,2023:06:24 14:28:18,$6.50,4.6,Chick-fil-A,,0,2023:06:24 14:01:32
3,$11.50,$0.00,$0.00,7-Eleven (23302) -,Yukiya Sushi -,$0.00,$11.50,2023:06:24 15:09:25,$11.50,10.8,Sushi Yukiya,7-Eleven,0,2023:06:24 14:32:10
4,$6.75,$0.00,$0.00,Starbucks (23194) -,N/A,N/A,$6.75,2023:06:24 15:42:26,$6.75,3.4,Starbucks,,0,2023:06:24 15:15:23
5,$5.75,$5.00,$0.00,Pizza on tap,N/A,N/A,$10.75,2023:06:24 16:20:22,$10.75,4.2,Pizza on tap,,0,2023:06:24 15:52:54
6,$5.00,$2.00,$0.00,Popeyes (1325),N/A,N/A,$7.00,2023:06:24 16:41:10,$7.00,3.7,Popeyes Louisiana Kitchen,,1,2023:06:24 15:55:57
7,$11.25,$8.50,$0.00,Upper Crust Pizza,N/A,N/A,$19.75,2023:06:24 17:17:55,$19.75,10.5,Upper Crust Pizza,,0,2023:06:24 16:49:34
8,$3.75,$1.00,$0.00,Wendy's (9076),N/A,N/A,$4.75,2023:06:24 18:11:10,$4.75,1.6,Wendy's,,0,2023:06:24 17:59:33
9,$6.75,$0.00,$0.00,Advanced Auto Parts (via OneRail) (3752 _,N/A,N/A,$6.75,2023:06:24 18:22:23,$6.75,1.5,Advance Auto Parts,,0,2023:06:24 18:11:48


In [544]:
#testOutputDF = totalDF.copy()
#testOutputDF

#So we need to split stacks, well split natty stacks, so step 1 is to flag stags
#Okay we have the stacked order set as a col, but it only has nat stacks not staggered
# So we need to....hmm
#How are nat stacks to separate addr oh right doordash just doesn't show the first one okay we're good
# So first we want to convert nat stacks to their own row
# If rest2 != N/A and Tip 2 != N/A, split
# Actually we can't split the math until we dedollar the info so
# First we convert, you know what lets just look for Rest2 != N/A
# That way we can set all values in tip 2 from N/A to 0.0
# Then, for "Base Pay", "Tip", "Peak Pay", "Tip 2", "Total", "Dollars" if $, strip to float
# Then for giving nat stacks their own row, divide base pay by 2 (round)
# We also want to grab the index of the thing it is stacked with
# Okay do this after snack time



#COORSTEST

In [545]:
testOutputDF = totalDF.copy()
def dollarToFloatCols(df, cols):
    for x in cols:
        s = df[x].items()
        for y in s:
            #First, look for any values in the given columns with N/A, replace with 0.0
            if("N/A" in y):
                df.at[y[0], x] = float(0.0)
            #Next look for errors that replaced dollar values with 0
            elif(y[1]==0):
                df.at[y[0], x] = float(0.0)
            #Next look for values that exited correctly with the $
            elif("$" in y[1]):
                df.at[y[0], x] = float(y[1][y[1].index("$")+1:])
            if not (isinstance(df.at[y[0], x], float)):
                print("ERROR AT:", y[0], x, type(df.at[y[0], x]))
                df.at[y[0], x] = float(0.0)
                print(df.at[y[0], x])
colsForFunc = ["Base Pay", "Tip", "Peak Pay", "Tip 2", "Total", "Dollars"]
#testOutputDF
dollarToFloatCols(testOutputDF, colsForFunc)
print(testOutputDF["Dollars"][20])
#testOutputDF

ERROR AT: 20 Dollars <class 'str'>
0.0
0.0


In [546]:
testOutputDF = testOutputDF.astype({"Base Pay": 'float', "Tip": 'float', "Peak Pay": 'float', 
                                    "Tip 2": 'float', "Total": 'float', "Dollars" : 'float'})

In [547]:
#testOutputDF
#So Restaurant Name looks a lot like raw data and RestNames looks a lot like restaurant names so
#We can have two dfs, one to update rawnames, and the other to update Deliveries

#Lets drop file names at this point

#testOutputDF = testOutputDF.drop(["Completed File Name", "Offer File Name"], axis=1)
testOutputDF

,Base Pay,Tip,Peak Pay,Restaurant Name,Restaurant Name 2,Tip 2,Total,Completed Timestamp,Dollars,Mileage,RestNames,Rest 2,Stacked Order,Offer Timestamp
0,6.50,7.0,0.0,China Fusion,N/A,0.0,13.50,2023:06:24 13:38:54,13.50,5.5,China Fusion,,0,2023:06:24 13:17:06
1,4.00,5.0,0.0,Hooked on Sushi (Carlsbad),N/A,0.0,9.00,2023:06:24 13:59:00,9.00,3.8,Hooked on Sushi,,0,2023:06:24 13:44:10
2,5.50,1.0,0.0,Chick-fil-A (01507),N/A,0.0,6.50,2023:06:24 14:28:18,6.50,4.6,Chick-fil-A,,0,2023:06:24 14:01:32
3,11.50,0.0,0.0,7-Eleven (23302) -,Yukiya Sushi -,0.0,11.50,2023:06:24 15:09:25,11.50,10.8,Sushi Yukiya,7-Eleven,0,2023:06:24 14:32:10
4,6.75,0.0,0.0,Starbucks (23194) -,N/A,0.0,6.75,2023:06:24 15:42:26,6.75,3.4,Starbucks,,0,2023:06:24 15:15:23
5,5.75,5.0,0.0,Pizza on tap,N/A,0.0,10.75,2023:06:24 16:20:22,10.75,4.2,Pizza on tap,,0,2023:06:24 15:52:54
6,5.00,2.0,0.0,Popeyes (1325),N/A,0.0,7.00,2023:06:24 16:41:10,7.00,3.7,Popeyes Louisiana Kitchen,,1,2023:06:24 15:55:57
7,11.25,8.5,0.0,Upper Crust Pizza,N/A,0.0,19.75,2023:06:24 17:17:55,19.75,10.5,Upper Crust Pizza,,0,2023:06:24 16:49:34
8,3.75,1.0,0.0,Wendy's (9076),N/A,0.0,4.75,2023:06:24 18:11:10,4.75,1.6,Wendy's,,0,2023:06:24 17:59:33
9,6.75,0.0,0.0,Advanced Auto Parts (via OneRail) (3752 _,N/A,0.0,6.75,2023:06:24 18:22:23,6.75,1.5,Advance Auto Parts,,0,2023:06:24 18:11:48


In [548]:
for index, row in testOutputDF.iterrows():
    if row["Stacked Order"] == 1:
        #print("STACK FOUND")
        testOutputDF.at[index-1, "Stacked Order"] = 1


for index, row in testOutputDF.iterrows():
    B_rest1 = row["RestNames"]
    B_rest2 = row["Rest 2"]
    A_rest1 = row["Restaurant Name"]
    A_rest2 = row["Restaurant Name 2"]
    

    
    #NAT STACK HANDLER
    if not B_rest2 == "":
        #print(B_rest2)
        errorFlag = False
        #### NEW ROW VARS ######
        
        newBase = row["Base Pay"]/2
        newTip = row["Tip 2"]
        newPeak = row["Peak Pay"]
        newRestaurantName = A_rest2
        newRestaurantName2 = ""
        newTotal = row["Total"]
        newTS_A = row["Completed Timestamp"]
        newDollars = row["Dollars"]
        newMileage = row["Mileage"]
        newRestName = ""
        newRest2 = ""
        newStackedFlag = 1
        newTS_B = row["Offer Timestamp"]
        
        ### OLD ROW VARS ######
        oldRestaurantName = A_rest1
        oldRestName = B_rest1
        oldRestName2 = oldRest2 = ""
        oldBase = row["Base Pay"]/2
        oldStackedFlag = 1
        
        
        
        
        
        
        # LOGIC SECTION
        if B_rest1 in A_rest1:
            #print("b1 in a1")
            newRestName = B_rest2
        elif B_rest1 in A_rest2:
            #print("b1 in a2")
            newRestName = B_rest1
            oldRestName = B_rest2
        elif B_rest2 in A_rest1:
            #print("b2 in a1")
            newRestName = B_rest1
            oldRestName = B_rest2
        elif B_rest2 in A_rest2:
            #print("b2 in a2")
            newRestName = B_rest2
        else:
            #print("Error")
            errorFlag = True
            newRestaurantName = "ERROR"
            newRestName = "ERROR"
            oldRestaurantName = "ERROR"
            oldRestName = "ERROR"
    
        #print("Restaurant Name for old row:", oldRestaurantName, "RestName for old row:", oldRestName)
        #Below we want to create the new row, update the old row, then sneak the new row into the df
        if not errorFlag:
            # NEW ROW #
            newIndex = index + 0.5
            testOutputDF.loc[newIndex] = newBase, newTip, newPeak, newRestaurantName, newRestaurantName2, 0.0, newTotal, newTS_A, newDollars, newMileage, newRestName, newRest2, newStackedFlag, newTS_B
            #testOutputDF = testOutputDF.sort_index().reset_index(drop=True)
            #print(newRestaurantName2, newRest2)    
 

            # OLD ROW #
            testOutputDF.at[index, "Restaurant Name"] = oldRestaurantName
            testOutputDF.at[index, "Restaurant Name 2"] = oldRestName2
            testOutputDF.at[index, "RestNames"] = oldRestName
            testOutputDF.at[index, "Rest 2"] = oldRest2
            testOutputDF.at[index, "Base Pay"] = oldBase
            testOutputDF.at[index, "Stacked Order"] = oldStackedFlag
            

            
#Once all vars stored locally, update old row, create new row, then we want to splice the row into the df
#or....index as index+.5?

#index = ah right we have that from iterrows
# newIndex = index+0.5
# df.loc[newIndex] = newRow
# df = df.sort_index().reset_index(drop=True)
testOutputDF = testOutputDF.sort_index().reset_index(drop=True)
display(testOutputDF)




#Something is wrong with b1 in a2 because its saving taco bell in old
#no no not saving taco bell in old, writing old to new

#okay the new row is keeping the old name2 and rest2 vals
#new row is chip tac tac chip
#and it doesn't do base/2, do some print statements tomorrow idk

#okay so I think its a problem with the way we handle splicing

#what we can try to fix it, is to have a list of new indeces, and a list of rowitems
#once the df has been iterrated through, we splice and reindex

#What I could also try real quick just in case it works, is to throw the thing outside the for loop 

#GOOD JOB ME, WE FOUND THE THING

,Base Pay,Tip,Peak Pay,Restaurant Name,Restaurant Name 2,Tip 2,Total,Completed Timestamp,Dollars,Mileage,RestNames,Rest 2,Stacked Order,Offer Timestamp
0,6.50,7.0,0.0,China Fusion,N/A,0.0,13.50,2023:06:24 13:38:54,13.50,5.5,China Fusion,,0,2023:06:24 13:17:06
1,4.00,5.0,0.0,Hooked on Sushi (Carlsbad),N/A,0.0,9.00,2023:06:24 13:59:00,9.00,3.8,Hooked on Sushi,,0,2023:06:24 13:44:10
2,5.50,1.0,0.0,Chick-fil-A (01507),N/A,0.0,6.50,2023:06:24 14:28:18,6.50,4.6,Chick-fil-A,,0,2023:06:24 14:01:32
3,5.75,0.0,0.0,7-Eleven (23302) -,,0.0,11.50,2023:06:24 15:09:25,11.50,10.8,7-Eleven,,1,2023:06:24 14:32:10
4,5.75,0.0,0.0,Yukiya Sushi -,,0.0,11.50,2023:06:24 15:09:25,11.50,10.8,Sushi Yukiya,,1,2023:06:24 14:32:10
5,6.75,0.0,0.0,Starbucks (23194) -,N/A,0.0,6.75,2023:06:24 15:42:26,6.75,3.4,Starbucks,,0,2023:06:24 15:15:23
6,5.75,5.0,0.0,Pizza on tap,N/A,0.0,10.75,2023:06:24 16:20:22,10.75,4.2,Pizza on tap,,1,2023:06:24 15:52:54
7,5.00,2.0,0.0,Popeyes (1325),N/A,0.0,7.00,2023:06:24 16:41:10,7.00,3.7,Popeyes Louisiana Kitchen,,1,2023:06:24 15:55:57
8,11.25,8.5,0.0,Upper Crust Pizza,N/A,0.0,19.75,2023:06:24 17:17:55,19.75,10.5,Upper Crust Pizza,,0,2023:06:24 16:49:34
9,3.75,1.0,0.0,Wendy's (9076),N/A,0.0,4.75,2023:06:24 18:11:10,4.75,1.6,Wendy's,,0,2023:06:24 17:59:33


In [549]:
#Okay now we have 14 items marked as stack and a length of 29
#We had 29 deliveries that day, 2 were nat stack, 5 were stag stacks, which brings us to where we want to be

In [550]:
exportDeliveriesDF = testOutputDF[["RestNames", "Total", "Base Pay", 
                                   "Tip", "Peak Pay", "Stacked Order", "Offer Timestamp", "Mileage",
                                  "Completed Timestamp"]].copy()
exportDeliveriesDF.head()

,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Offer Timestamp,Mileage,Completed Timestamp
0,China Fusion,13.5,6.50,7.0,0.0,0,2023:06:24 13:17:06,5.5,2023:06:24 13:38:54
1,Hooked on Sushi,9.0,4.00,5.0,0.0,0,2023:06:24 13:44:10,3.8,2023:06:24 13:59:00
2,Chick-fil-A,6.5,5.50,1.0,0.0,0,2023:06:24 14:01:32,4.6,2023:06:24 14:28:18
3,7-Eleven,11.5,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25
4,Sushi Yukiya,11.5,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25


In [551]:
dateFromDF = exportDeliveriesDF["Offer Timestamp"][0]
#dateFromDF = dateFromDF[:dateFromDF.index(" ")]
#We can hard code values since we know its always going to be yyyy:mm:dd
#Actually we don't even need to peel the time off
dateFromDF = dateFromDF[5:10] + "/" + dateFromDF[0:4]
dateFromDF = dateFromDF.replace(":", "/")
#print(dateFromDF)
exportDeliveriesDF["Date"] = dateFromDF
#Honestly we can just set the date to whatever the first col value is
exportDeliveriesDF.head()

,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Offer Timestamp,Mileage,Completed Timestamp,Date
0,China Fusion,13.5,6.50,7.0,0.0,0,2023:06:24 13:17:06,5.5,2023:06:24 13:38:54,06/24/2023
1,Hooked on Sushi,9.0,4.00,5.0,0.0,0,2023:06:24 13:44:10,3.8,2023:06:24 13:59:00,06/24/2023
2,Chick-fil-A,6.5,5.50,1.0,0.0,0,2023:06:24 14:01:32,4.6,2023:06:24 14:28:18,06/24/2023
3,7-Eleven,11.5,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023
4,Sushi Yukiya,11.5,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023


In [552]:
#Again, since we know the format is always yyyy:mm:dd hh:mm:ss
#Not sure how Sheets will convert, as is I type in 24 hour time and it displays in AM/PM 
#Lets just convert to am/pm for now
exportDeliveriesDF["Start Time"] = 1
exportDeliveriesDF["End Time"] = 1
for index, value in exportDeliveriesDF.iterrows():
    startTime = exportDeliveriesDF["Offer Timestamp"][index]
    startTime = startTime[11:16]
    endTime = exportDeliveriesDF["Completed Timestamp"][index]
    endTime = endTime[11:16]
    #CHANGE THIS TO ZERO IF SHEETS ENDS UP BEING SILLY LOOKING
    convertTo12HrFlag = 1
    if(convertTo12HrFlag):
        intA = int(startTime[0:2])
        intB = int(endTime[0:2])
        if(intA >= 13):
            intA = intA-12
        if(intB >= 13):
            intB = intB-12
        strA = str(intA)
        strB = str(intB)
        startTime = strA + startTime[2:5]
        endTime = strB + endTime[2:5]
        #print(startTime, endTime)
        exportDeliveriesDF.at[index, "Start Time"] = startTime
        exportDeliveriesDF.at[index, "End Time"] = endTime
    
    
exportDeliveriesDF

,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Offer Timestamp,Mileage,Completed Timestamp,Date,Start Time,End Time
0,China Fusion,13.50,6.50,7.0,0.0,0,2023:06:24 13:17:06,5.5,2023:06:24 13:38:54,06/24/2023,1:17,1:38
1,Hooked on Sushi,9.00,4.00,5.0,0.0,0,2023:06:24 13:44:10,3.8,2023:06:24 13:59:00,06/24/2023,1:44,1:59
2,Chick-fil-A,6.50,5.50,1.0,0.0,0,2023:06:24 14:01:32,4.6,2023:06:24 14:28:18,06/24/2023,2:01,2:28
3,7-Eleven,11.50,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023,2:32,3:09
4,Sushi Yukiya,11.50,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023,2:32,3:09
5,Starbucks,6.75,6.75,0.0,0.0,0,2023:06:24 15:15:23,3.4,2023:06:24 15:42:26,06/24/2023,3:15,3:42
6,Pizza on tap,10.75,5.75,5.0,0.0,1,2023:06:24 15:52:54,4.2,2023:06:24 16:20:22,06/24/2023,3:52,4:20
7,Popeyes Louisiana Kitchen,7.00,5.00,2.0,0.0,1,2023:06:24 15:55:57,3.7,2023:06:24 16:41:10,06/24/2023,3:55,4:41
8,Upper Crust Pizza,19.75,11.25,8.5,0.0,0,2023:06:24 16:49:34,10.5,2023:06:24 17:17:55,06/24/2023,4:49,5:17
9,Wendy's,4.75,3.75,1.0,0.0,0,2023:06:24 17:59:33,1.6,2023:06:24 18:11:10,06/24/2023,5:59,6:11


In [561]:
exportDeliveriesDF["Duration"] = 0
for index, value in exportDeliveriesDF.iterrows():
    #check lengths first
    #if length = 4 do x, if 5 do y
    endInt = value["End Time"]
    startInt = value["Start Time"]
    if(len(endInt) == 4):
        endInt = int(endInt[2:5])
    else:
        endInt = int(endInt[3:6])    
    if(len(startInt) == 4):
        startInt = int(startInt[2:5])
    else:
        startInt = int(startInt[3:6])
    
    #print(endInt, startInt)
    durVal = endInt - startInt
    if (durVal <= 0):
        durVal = endInt + 60 - startInt
    #print(durVal)
    exportDeliveriesDF.at[index, "Duration"] = durVal
#exportDeliveriesDF

In [562]:
exportDeliveriesDF

#We could grab in the latest value for ID from the Deliveries sheet, or we could just drag the cell down in sheets
#If we drag the cell formula down for ID, the only thing we need to manually update here is stacked order ID

#So yeah, lets grab the most recent ID, well lets just make it a enter for now no thats lazy
#We grab the -1 from Deliveries[ID], reindex with that+1, then if Stacked Order, use index
#After that we re $ Total, Base Pay, Tip, Peak Pay, and drop unwanted columns and its good. This is a project
#that can be completed when it isn't 3am

,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Offer Timestamp,Mileage,Completed Timestamp,Date,Start Time,End Time,Duration
0,China Fusion,13.50,6.50,7.0,0.0,0,2023:06:24 13:17:06,5.5,2023:06:24 13:38:54,06/24/2023,1:17,1:38,21
1,Hooked on Sushi,9.00,4.00,5.0,0.0,0,2023:06:24 13:44:10,3.8,2023:06:24 13:59:00,06/24/2023,1:44,1:59,15
2,Chick-fil-A,6.50,5.50,1.0,0.0,0,2023:06:24 14:01:32,4.6,2023:06:24 14:28:18,06/24/2023,2:01,2:28,27
3,7-Eleven,11.50,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023,2:32,3:09,37
4,Sushi Yukiya,11.50,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023,2:32,3:09,37
5,Starbucks,6.75,6.75,0.0,0.0,0,2023:06:24 15:15:23,3.4,2023:06:24 15:42:26,06/24/2023,3:15,3:42,27
6,Pizza on tap,10.75,5.75,5.0,0.0,1,2023:06:24 15:52:54,4.2,2023:06:24 16:20:22,06/24/2023,3:52,4:20,28
7,Popeyes Louisiana Kitchen,7.00,5.00,2.0,0.0,1,2023:06:24 15:55:57,3.7,2023:06:24 16:41:10,06/24/2023,3:55,4:41,46
8,Upper Crust Pizza,19.75,11.25,8.5,0.0,0,2023:06:24 16:49:34,10.5,2023:06:24 17:17:55,06/24/2023,4:49,5:17,28
9,Wendy's,4.75,3.75,1.0,0.0,0,2023:06:24 17:59:33,1.6,2023:06:24 18:11:10,06/24/2023,5:59,6:11,12


In [553]:
rawDataDF = testOutputDF[["RestNames", "Restaurant Name"]].copy()
rawDataDF

,RestNames,Restaurant Name
0,China Fusion,China Fusion
1,Hooked on Sushi,Hooked on Sushi (Carlsbad)
2,Chick-fil-A,Chick-fil-A (01507)
3,7-Eleven,7-Eleven (23302) -
4,Sushi Yukiya,Yukiya Sushi -
5,Starbucks,Starbucks (23194) -
6,Pizza on tap,Pizza on tap
7,Popeyes Louisiana Kitchen,Popeyes (1325)
8,Upper Crust Pizza,Upper Crust Pizza
9,Wendy's,Wendy's (9076)


In [573]:
#So that's an interesting thing about stacks, for stag stacks, it can go pickup a, pickup b, dropoff b, dropoff a

#so index(restname) in restaurant name (for stacked orders)
#no, if restname in restaurant name, rest1 = rest1, newrowrest = rest 2
# else, newrowrest = rest 1

#Go through testoutputdf where stacked order = 1
#Check if restnames in restaurant name, if not, print both
#then, if restnames not in restaurantname, print next index restaurantname and restnames
#if restnames_1 in restaurantname_2 then swap

#In theory we can get a triple stack, but for now just auto decline them
testDFX = testOutputDF.loc[testOutputDF["Stacked Order"] == 1]
testDFX


,Base Pay,Tip,Peak Pay,Restaurant Name,Restaurant Name 2,Tip 2,Total,Completed Timestamp,Dollars,Mileage,RestNames,Rest 2,Stacked Order,Offer Timestamp
3,5.75,0.0,0.0,7-Eleven (23302) -,,0.0,11.50,2023:06:24 15:09:25,11.50,10.8,7-Eleven,,1,2023:06:24 14:32:10
4,5.75,0.0,0.0,Yukiya Sushi -,,0.0,11.50,2023:06:24 15:09:25,11.50,10.8,Sushi Yukiya,,1,2023:06:24 14:32:10
6,5.75,5.0,0.0,Pizza on tap,N/A,0.0,10.75,2023:06:24 16:20:22,10.75,4.2,Pizza on tap,,1,2023:06:24 15:52:54
7,5.00,2.0,0.0,Popeyes (1325),N/A,0.0,7.00,2023:06:24 16:41:10,7.00,3.7,Popeyes Louisiana Kitchen,,1,2023:06:24 15:55:57
15,2.25,0.0,0.0,Taverna Mediterranean Cuisine (East Vista,N/A,0.0,8.25,2023:06:24 20:57:31,5.50,3.7,Kung Fu Noodle,,1,2023:06:24 20:33:31
16,5.50,0.0,0.0,Kung Fu Noodle -,N/A,0.0,5.50,2023:06:24 21:01:55,6.25,0.6,Taverna Mediterranean Cuisine,,1,2023:06:24 20:35:50
18,4.25,3.5,0.0,Chipotle (Mission Marketplace),,5.0,17.00,2023:06:24 22:18:22,17.00,4.1,Chipotle,,1,2023:06:24 21:48:28
19,4.25,5.0,0.0,Taco Bell (40797),,0.0,17.00,2023:06:24 22:18:22,17.00,4.1,Taco Bell,,1,2023:06:24 21:48:28
21,0.00,0.0,0.0,0,0,0.0,0.00,2023:06:24 23:01:32,7.75,3.6,7-Eleven,,1,2023:06:24 22:43:40
22,3.00,3.0,0.0,Pizza Hut (027196) Marketplace - Summit,"Pizza West, LLC :",0.0,6.00,2023:06:24 23:06:54,0.00,1.7,ERROR,ERROR,1,2023:06:24 22:46:59


In [575]:
#Okay at some point the error check was to set restaurant name to zero instead of N/A, fix that later
#in the mean time, will cast to str

#POPEYES WHY
#Okay I'm not at all sure how to go about working through checking for if
#Okay well I am but its not worth the process I dont think, easier to handle errors manually
#Would need to split RestNames up into list of strings, with regex maybe, then check for 
#no that wouldn't even work because a lot of places have things like cuisine or restaurant or prepositions
#Yeah we can handle it manually
for index, value in testDFX.iterrows():
    if not(str(value["RestNames"]) in str(value["Restaurant Name"])):
        print(index, value["Restaurant Name"], value["RestNames"])
        
#We can left join on index and export to sheets, just copy two columns to rawdata and the rest to deliveries
#After hand checking

4 Yukiya Sushi - Sushi Yukiya
7 Popeyes (1325)  Popeyes Louisiana Kitchen
15 Taverna Mediterranean Cuisine (East Vista Kung Fu Noodle
16 Kung Fu Noodle - Taverna Mediterranean Cuisine
21 0 7-Eleven
22 Pizza Hut (027196) Marketplace - Summit  ERROR


In [583]:
import pygsheets
gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')
sheets_DelDF       = sh[1].get_as_df()
newIndexForExport = sheets_DelDF["ID"].iloc[-1] + 1

In [593]:
exportDeliveriesDF

,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Offer Timestamp,Mileage,Completed Timestamp,Date,Start Time,End Time,Duration,ID
0,China Fusion,13.50,6.50,7.0,0.0,0,2023:06:24 13:17:06,5.5,2023:06:24 13:38:54,06/24/2023,1:17,1:38,21,886
1,Hooked on Sushi,9.00,4.00,5.0,0.0,0,2023:06:24 13:44:10,3.8,2023:06:24 13:59:00,06/24/2023,1:44,1:59,15,887
2,Chick-fil-A,6.50,5.50,1.0,0.0,0,2023:06:24 14:01:32,4.6,2023:06:24 14:28:18,06/24/2023,2:01,2:28,27,888
3,7-Eleven,11.50,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023,2:32,3:09,37,889
4,Sushi Yukiya,11.50,5.75,0.0,0.0,1,2023:06:24 14:32:10,10.8,2023:06:24 15:09:25,06/24/2023,2:32,3:09,37,890
5,Starbucks,6.75,6.75,0.0,0.0,0,2023:06:24 15:15:23,3.4,2023:06:24 15:42:26,06/24/2023,3:15,3:42,27,891
6,Pizza on tap,10.75,5.75,5.0,0.0,1,2023:06:24 15:52:54,4.2,2023:06:24 16:20:22,06/24/2023,3:52,4:20,28,892
7,Popeyes Louisiana Kitchen,7.00,5.00,2.0,0.0,1,2023:06:24 15:55:57,3.7,2023:06:24 16:41:10,06/24/2023,3:55,4:41,46,893
8,Upper Crust Pizza,19.75,11.25,8.5,0.0,0,2023:06:24 16:49:34,10.5,2023:06:24 17:17:55,06/24/2023,4:49,5:17,28,894
9,Wendy's,4.75,3.75,1.0,0.0,0,2023:06:24 17:59:33,1.6,2023:06:24 18:11:10,06/24/2023,5:59,6:11,12,895


In [620]:
#Also this is going to be wrong since recent orders were done by hand
#once the ID is set, we want to go through the DF and
#If stackedorder=1, update stackedorder to be ID+1, update next index stacked order to be ID, increment index by 1
# -----as long as not out of bounds

ID_array = np.arange(newIndexForExport, newIndexForExport+len(exportDeliveriesDF))
print(newIndexForExport, len(exportDeliveriesDF))
print(len(ID_array))
exportDeliveriesDF["ID"] = ID_array
exportDeliveriesDF_1 = exportDeliveriesDF[["ID", "Date", "RestNames", "Total", "Base Pay", 
                                          "Tip", "Peak Pay", "Stacked Order", "Start Time", "Mileage", 
                                          "End Time", "Duration"]].copy()
exportDeliveriesDF_1

886 29
29


,ID,Date,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Start Time,Mileage,End Time,Duration
0,886,06/24/2023,China Fusion,13.50,6.50,7.0,0.0,0,1:17,5.5,1:38,21
1,887,06/24/2023,Hooked on Sushi,9.00,4.00,5.0,0.0,0,1:44,3.8,1:59,15
2,888,06/24/2023,Chick-fil-A,6.50,5.50,1.0,0.0,0,2:01,4.6,2:28,27
3,889,06/24/2023,7-Eleven,11.50,5.75,0.0,0.0,1,2:32,10.8,3:09,37
4,890,06/24/2023,Sushi Yukiya,11.50,5.75,0.0,0.0,1,2:32,10.8,3:09,37
5,891,06/24/2023,Starbucks,6.75,6.75,0.0,0.0,0,3:15,3.4,3:42,27
6,892,06/24/2023,Pizza on tap,10.75,5.75,5.0,0.0,1,3:52,4.2,4:20,28
7,893,06/24/2023,Popeyes Louisiana Kitchen,7.00,5.00,2.0,0.0,1,3:55,3.7,4:41,46
8,894,06/24/2023,Upper Crust Pizza,19.75,11.25,8.5,0.0,0,4:49,10.5,5:17,28
9,895,06/24/2023,Wendy's,4.75,3.75,1.0,0.0,0,5:59,1.6,6:11,12


In [621]:
exportDeliveriesDF_1["Version"] = 3

In [623]:
exportDelDF = exportDeliveriesDF_1.loc[exportDeliveriesDF_1["Stacked Order"] == 1]
for index, value in exportDelDF.iterrows():
    if (value["Stacked Order"] == 1):
        print(value["ID"], value["ID"]+1, value["Stacked Order"])
        if(index < len(exportDelDF)-1):
            #print(value["ID"], value["ID"]+1, value["Stacked Order"])
            #print("Swap")
            exportDelDF.at[index, "Stacked Order"] = value["ID"]+1
            exportDelDF.at[index+1, "Stacked Order"] = value["ID"]
#exportDeliveriesDF_1
#I may be too tired to notice if this is overthinking the problem or underthinking the problem


889 890 1
890 891 1
892 893 1
893 894 1
901 902 1
902 903 1
904 905 1
905 906 1
907 908 1
908 909 1
910 911 1
911 912 1
913 914 1
914 915 1


/tmp/ipykernel_7900/722854084.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exportDelDF.at[index+1, "Stacked Order"] = value["ID"]


,ID,Date,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Start Time,Mileage,End Time,Duration,Version
0,886,06/24/2023,China Fusion,13.50,6.50,7.0,0.0,0,1:17,5.5,1:38,21,3
1,887,06/24/2023,Hooked on Sushi,9.00,4.00,5.0,0.0,0,1:44,3.8,1:59,15,3
2,888,06/24/2023,Chick-fil-A,6.50,5.50,1.0,0.0,0,2:01,4.6,2:28,27,3
3,889,06/24/2023,7-Eleven,11.50,5.75,0.0,0.0,1,2:32,10.8,3:09,37,3
4,890,06/24/2023,Sushi Yukiya,11.50,5.75,0.0,0.0,1,2:32,10.8,3:09,37,3
5,891,06/24/2023,Starbucks,6.75,6.75,0.0,0.0,0,3:15,3.4,3:42,27,3
6,892,06/24/2023,Pizza on tap,10.75,5.75,5.0,0.0,1,3:52,4.2,4:20,28,3
7,893,06/24/2023,Popeyes Louisiana Kitchen,7.00,5.00,2.0,0.0,1,3:55,3.7,4:41,46,3
8,894,06/24/2023,Upper Crust Pizza,19.75,11.25,8.5,0.0,0,4:49,10.5,5:17,28,3
9,895,06/24/2023,Wendy's,4.75,3.75,1.0,0.0,0,5:59,1.6,6:11,12,3


In [627]:
secondHalf = False
for index, value in exportDeliveriesDF_1.iterrows():
    if secondHalf:
        secondHalf = False
        exportDeliveriesDF_1.at[index, "Stacked Order"] = value["ID"] - 1
    elif(value["Stacked Order"] == 1):
        secondHalf = True
        exportDeliveriesDF_1.at[index, "Stacked Order"] = value["ID"] + 1
exportDeliveriesDF_1

889
890
892
893
901
902
904
905
907
908
910
911
913
914


,ID,Date,RestNames,Total,Base Pay,Tip,Peak Pay,Stacked Order,Start Time,Mileage,End Time,Duration,Version
0,886,06/24/2023,China Fusion,13.50,6.50,7.0,0.0,0,1:17,5.5,1:38,21,3
1,887,06/24/2023,Hooked on Sushi,9.00,4.00,5.0,0.0,0,1:44,3.8,1:59,15,3
2,888,06/24/2023,Chick-fil-A,6.50,5.50,1.0,0.0,0,2:01,4.6,2:28,27,3
3,889,06/24/2023,7-Eleven,11.50,5.75,0.0,0.0,890,2:32,10.8,3:09,37,3
4,890,06/24/2023,Sushi Yukiya,11.50,5.75,0.0,0.0,889,2:32,10.8,3:09,37,3
5,891,06/24/2023,Starbucks,6.75,6.75,0.0,0.0,0,3:15,3.4,3:42,27,3
6,892,06/24/2023,Pizza on tap,10.75,5.75,5.0,0.0,893,3:52,4.2,4:20,28,3
7,893,06/24/2023,Popeyes Louisiana Kitchen,7.00,5.00,2.0,0.0,892,3:55,3.7,4:41,46,3
8,894,06/24/2023,Upper Crust Pizza,19.75,11.25,8.5,0.0,0,4:49,10.5,5:17,28,3
9,895,06/24/2023,Wendy's,4.75,3.75,1.0,0.0,0,5:59,1.6,6:11,12,3


In [652]:
#gc = pygsheets.authorize(service_file='file.json')
#sh = gc.open('DashData')
rawDataDF = rawDataDF.rename(columns={"RestNames":"Name", "Restaurant Name": "raw data"})
exportDeliveriesDF_1 = exportDeliveriesDF_1.join(rawDataDF)
rawDataDF
wks = sh[7]
wks.set_dataframe(exportDeliveriesDF_1,(1,1))

In [ ]:
print("Awkward")

In [634]:
#OKAY SO FOR SOME REASON, ALMOST ALL THE IDS HAVE A DOLLAR SIGN, 3 OF THE TIPS DO, 3 OF THE PEAK PAY DONT
#WHICH IS WEIRD BECAUSE I AM LOOKING RIGHT AT THE DATAFRAME
exportDeliveriesDF_1.dtypes
#YEAH TIP AND PEAK ARE FLOATS SO, WHY?

#welp, lets try it this way then, convert all columns to string for export


ID                object
Date              object
RestNames         object
Total            float64
Base Pay         float64
Tip              float64
Peak Pay         float64
Stacked Order     object
Start Time        object
Mileage           object
End Time          object
Duration          object
Version           object
dtype: object

In [650]:
exportDeliveriesDF_1 = exportDeliveriesDF_1.applymap(str)
wks.set_dataframe(exportDeliveriesDF_1,(1,1))

In [ ]:
#There's my sanity restored
#We actually will want to convert total, base, tip, peak to $##.## format
#and likely want to convert 12 hr to 24 hr

#Unless sheets has a paste format option, which I think it does
#It does but its not what we want
#easy enough to select 4 sheets columns and hit currency format
#Look through pygsheets docs and see if formatting is a thing


#Okay so can export, then look here https://pygsheets.readthedocs.io/en/stable/examples.html
#FormatType is an enum, use CURRENCY instead of PERCENT
#There is also a formattype for time

#I think the process goes like this
#select a cell to be used, maybe Cell isn't actually a cell but something that could be, an A1 is the label?
# model_cell = pygsheets.Cell("A1")
# model_cell.set_number_format(format_type = pygsheets.FormatType.CURRENCY) #pattern="0%" ?? pattern="$0.00" ?
# pygsheets.DataRange("D2", "F30", worksheet = wks).apply_format(modal_cell) #might have to use start=d2, end=f30
# ------------------and f30 isn't always going to be it, like it'll be f+str(len(df)+1)
#

#then I and K but not J, so just I2 - I30 and K2-K30 for FormatType.TIME

#https://pygsheets.readthedocs.io/en/stable/examples.html
#https://pygsheets.readthedocs.io/en/stable/_modules/pygsheets/custom_types.html?highlight=formattype
#https://pygsheets.readthedocs.io/en/stable/datarange.html?highlight=datarange
#https://pygsheets.readthedocs.io/en/stable/cell.html?highlight=cell

In [641]:
#So the last row is going to be len(df)+1, because the attributes are posted in the top row on sheets
#lastRowOfSheets = len(exportDeliveriesDF_1)+1
#lastRowOfSheets = "G" + str(lastRowOfSheets)
#lastRowOfSheets

'G30'

In [647]:


#model_cell = pygsheets.Cell("A1")
#model_cell.set_number_format(format_type = pygsheets.FormatType.CURRENCY,
#                            pattern="$0.00") 
#pattern="0%" ?? pattern="$0.00" ?
#pygsheets.DataRange(start="D2", end=lastRowOfSheets, worksheet = wks).apply_format(model_cell)


#Well, the pygsheets docs examples dont seem to work, and I can't seem to get this to format to currency
#examplem was percentage. It seems to be applying something to the correct range, but the format is unchanged

#Maybe if we just set the format in sheets, throwing the DF on top will work correctly

In [ ]:
#That seems to work, so lets join  rawDataDF and see what happens

In [ ]:
#Okay so, next step is making this sheet not look like a scene out of Oppenheimer